In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:/PREM\MACHINE LEARNING/Deep Learning/RNN/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [3]:
## Drop Nan values
df = df.dropna()
df.drop('id',axis=1,inplace=True)
df.shape

C:\Users\Prem Ranjan\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(18285, 4)

In [4]:
## Get the Independent & Dependent Features
X = df.drop('label',axis=1)
Y = df['label']
X.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

In [6]:
## Vocabulary size
voc_size = 5000

## One-Hot Representation

In [7]:
messages = X.copy()

In [8]:
messages.reset_index(inplace=True)

In [9]:
import nltk
import re
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Prem
[nltk_data]     Ranjan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
print(corpus[0:5])
print('Length of corpus : ',len(corpus))

['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire', 'civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri']
Length of corpus :  18285


In [13]:
# one hot encoding
onehot_rep = [one_hot(word ,voc_size) for word in corpus]
onehot_rep[0:5]

[[806, 4162, 2341, 3950, 1662, 4231, 1250, 2705, 2842, 3328],
 [2836, 4896, 50, 4037, 466, 1706, 896],
 [3359, 433, 2330, 4625],
 [2584, 2734, 1512, 2145, 4738, 3175],
 [4223, 466, 1553, 769, 1227, 2516, 466, 4491, 4210, 27]]

In [14]:
## adding padding to make the list of same size
sent_len = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_len)
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  806,
       4162, 2341, 3950, 1662, 4231, 1250, 2705, 2842, 3328])

In [15]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
## converting X and Y to array
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [17]:
## Spliting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

## Model Training

In [18]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 37ms/step - loss: 0.3342 - accuracy: 0.8408 - val_loss: 0.2006 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1378 - accuracy: 0.9456 - val_loss: 0.1903 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0954 - accuracy: 0.9652 - val_loss: 0.2175 - val_accuracy: 0.9171
Epoch 4/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0694 - accuracy: 0.9755 - val_loss: 0.2957 - val_accuracy: 0.9186
Epoch 5/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0448 - accuracy: 0.9855 - val_loss: 0.3004 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0311 - accuracy: 0.9902 - val_loss: 0.3767 - val_accuracy: 0.9138
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0192 - accuracy: 0.9937 - val_loss: 0.4213 - val_accuracy: 0.9112

### Performance metrics and Accuracy

In [19]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [23]:
confusion_matrix(y_test, y_pred)

array([[3131,  288],
       [ 277, 2339]], dtype=int64)

In [25]:
accuracy_score(y_test, y_pred)

0.9063794531897266

### Adding dropout to increase accuracy

In [26]:
from tensorflow.keras.layers import Dropout

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 37ms/step - loss: 0.3384 - accuracy: 0.8399 - val_loss: 0.1929 - val_accuracy: 0.9170
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1504 - accuracy: 0.9407 - val_loss: 0.1923 - val_accuracy: 0.9244
Epoch 3/10
192/192 [==============================] - 6s 29ms/step - loss: 0.1131 - accuracy: 0.9575 - val_loss: 0.2032 - val_accuracy: 0.9206
Epoch 4/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0899 - accuracy: 0.9673 - val_loss: 0.2460 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0717 - accuracy: 0.9754 - val_loss: 0.2657 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0548 - accuracy: 0.9822 - val_loss: 0.2937 - val_accuracy: 0.9112
Epoch 7/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0450 - accuracy: 0.9847 - val_loss: 0.3824 - val_accuracy: 0.9150

In [29]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_test, y_pred)

array([[3128,  291],
       [ 278, 2338]], dtype=int64)

In [30]:
accuracy_score(y_test, y_pred)

0.9057166528583265

There is no increase in accuracy when using dropout!